In [57]:
import pandas as pd

In [58]:
df = pd.read_csv('http://bit.ly/kaggletrain')
#Using the titanic dataset

In [59]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [60]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [61]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [62]:
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]

In [63]:
df.shape

(889, 4)

In [64]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Embarked    0
dtype: int64

In [65]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [66]:
X = df.loc[:, ['Pclass']]
y = df.Survived

In [67]:
X.shape

(889, 1)

In [68]:
y.shape

(889,)

In [69]:
from sklearn.linear_model import LogisticRegression

In [70]:
logreg = LogisticRegression(solver = 'lbfgs') #Solver is the way we use our cost function see https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions

In [71]:
from sklearn.model_selection import cross_val_score

In [72]:
cross_val_score(logreg, X, y, cv=2, scoring='accuracy').mean()

0.6783227047271991

In [73]:
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

In [74]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In order to create a better <b> Cross validation</b> (Train test Split) process. We will have to <b>encode </b> the <b>Unordered categorical features </b> by using <b>DummyEncoding</b> or <b>OneHotEncoding</b>. 

In [75]:
from sklearn.preprocessing import OneHotEncoder
ohe_sex = OneHotEncoder(sparse=False) 
ohe_ebmarked = OneHotEncoder(sparse=False)
# space will return the sparse matrix datatype
# we want a regular numpy array

In [76]:
ohe_sex.fit_transform(df[['Sex']])

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [77]:
ohe_sex.categories_ #This is the categories, that we encoding to from our Sex column

[array(['female', 'male'], dtype=object)]

In [78]:
ohe_ebmarked.fit_transform(df[['Embarked']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [79]:
ohe_ebmarked.categories_

[array(['C', 'Q', 'S'], dtype=object)]

It is also possible to do implicat OHE using pandas, but we will usre the SKlearn version here

In [80]:
#lets refit the model

In [81]:
X = df.drop('Survived', axis='columns')

In [82]:
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [83]:
from sklearn.compose import make_column_transformer

In [87]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Sex', 'Embarked']),
    remainder='passthrough')


#With the colunmn Transformer we pass an instance of our OneHotEncoder class, 
#The columns we want to encode, 
#The last paranmeter is, the strategy to manipulate, with the rest of the features. 
#In this case we want them to <i>passthrough</i>, so we want mutate them

In [86]:
column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [88]:
from sklearn.pipeline import make_pipeline

In [89]:
pipe = make_pipeline(column_trans, logreg)

In [90]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

The line above means, that an entire pipeline is being <b> Cross Validated </b> Not just the model like before. 
This means that:
<li> First the data is cross-validated and split into <i>X</i> and <i>Y</i>
<li> After that the Pipeline is run to perform OneHotEncoding
<li> Model is built and trained

In [95]:
#Built test data to train new model
#Sample data
X_new = X.sample(5, random_state=100)
X_new.head(3)

,Pclass,Sex,Embarked
639,3,male,S
147,3,female,S
278,3,male,Q


In [96]:
#use the pipeline to fit the data 

In [97]:
pipe.fit(X, y)

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                n_values=None,
                                                                sparse=True),
                                                  ['Sex', 'Embarked'])],
                      

In [99]:
pipe.predict(X_new)

array([0, 1, 0, 1, 0], dtype=int64)

In [102]:
logreg.predict(X_new) #Will fail because we have not applied the pipeline yet. 

ValueError: could not convert string to float: 'male'